In [5]:
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.constants import AssetTypes

# 0. CONNECT
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    print("DefaultAzureCredential failed; falling back to InteractiveBrowserCredential...")
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential, path="../config.json")
print(f"Workspace: {ml_client.workspace_name}")

# 1. GET LATEST ENRICHED ASSET (this must be the version you just registered today)
enriched_asset_latest = ml_client.data.get(
    name="diabetes-clinical-enriched-130us",
    version="1127_1308",
)
print("Using enriched asset:", enriched_asset_latest.name, "version:", enriched_asset_latest.version)

training_data_input = Input(
    type=AssetTypes.MLTABLE,
    path=f"azureml:{enriched_asset_latest.name}:{enriched_asset_latest.version}",
)

TARGET_COL = "readmitted_30d_binary"  # binary: 1 = <30, 0 = NO or >30

# 2. DEFINE BINARY AUTOML CLASSIFICATION JOB
from azure.ai.ml import automl

automl_job = automl.classification(
    compute="clinical-cluster-cpu",
    experiment_name="Clinical_Readmission_AutoML_Binary_Clean",
    training_data=training_data_input,
    target_column_name=TARGET_COL,
    primary_metric="AUC_weighted",
)

automl_job.set_limits(
    timeout_minutes=60,
    max_trials=20,
    max_concurrent_trials=3,
)

automl_job.display_name = "Clinical_Readmission_AutoML_Binary_Final"

# 3. SUBMIT
returned_automl = ml_client.jobs.create_or_update(automl_job)
print("AutoML job submitted:")
print(returned_automl.studio_url)


Found the config file in: ..\config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Workspace: AML-Clinical-Readmission
Using enriched asset: diabetes-clinical-enriched-130us version: 1127_1308
AutoML job submitted:
https://ml.azure.com/runs/bright_net_hf5lpcd5vc?wsid=/subscriptions/3aeb63fe-f831-47f0-8175-3732f2efd2a1/resourcegroups/RG-Clinical-Readmission/workspaces/AML-Clinical-Readmission&tid=deb5bf9d-8bb0-4783-8f54-42a424392492
